<a href="https://colab.research.google.com/github/mariyajoseph2002/ml/blob/main/deepr2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving healpal_mood_risk_dataset_5000.csv to healpal_mood_risk_dataset_5000.csv


In [ ]:
df = pd.read_csv("healpal_mood_risk_dataset.csv")
X = df.drop(columns=["risk_level"])  # Features
y = df["risk_level"]  # Target labels (0, 1, 2, 3)


In [ ]:
scaler = StandardScaler()
X["mood_trend"] = scaler.fit_transform(X[["mood_trend"]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
#no need oof this
y_resampled = to_categorical(y_resampled, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [ ]:
model = tf.keras.Sequential([
    Dense(16, activation="relu", input_shape=(X.shape[1],)),  # Input layer
    Dense(8, activation="relu"),  # Hidden layer
    Dense(4, activation="softmax")  # Output layer (4 risk levels)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
model.fit(X_resampled, y_resampled, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2707 - loss: 1.6865 - val_accuracy: 0.1900 - val_loss: 1.5997
Epoch 2/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2537 - loss: 1.4770 - val_accuracy: 0.2100 - val_loss: 1.4141
Epoch 3/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2718 - loss: 1.4061 - val_accuracy: 0.2900 - val_loss: 1.3332
Epoch 4/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3057 - loss: 1.3508 - val_accuracy: 0.3700 - val_loss: 1.2668
Epoch 5/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3795 - loss: 1.2881 - val_accuracy: 0.4200 - val_loss: 1.2027
Epoch 6/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4635 - loss: 1.2388 - val_accuracy: 0.5400 - val_loss: 1.1243
Epoch 7/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5461 - loss: 1.1833 - val_accuracy: 0.6100 - val_loss: 1.0198
Epoch 8/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5890 - loss: 1.1019 - val_accuracy: 0.5400 - val_loss

In [ ]:
feature_means = scaler.mean_
feature_stds = np.sqrt(scaler.var_)  # Standard deviation

print("Means:", feature_means)
print("Standard Deviations:", feature_stds)

Means: [41.05714286]
Standard Deviations: [17.76331923]


In [ ]:
model.save("depression_model2.h5")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

Saved artifact at '/tmp/tmpe7hvl8jp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 8), dtype=tf.float32, name='keras_tensor_4')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  136247416468112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247416468688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247416468880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247416470032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247416469456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136247416470800: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
with open("depression_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved ")


TFLite model saved 


In [ ]:


files.download("depression_model.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
new_data = pd.DataFrame({
    "mood": [2],  # 😶 Numb (3)
    "social_interaction": [1],  # No, alone (1)
    "work_productivity": [0],  # 🤯 Struggled (3)
    "hobbies_selfcare": [2],  # 😕 No (3)
    "emotional_triggers": [1],  # Slight issues (1)
    "sleep_quality": [0],  # 😐 Fair (1)
    "appetite":[0],
    "mood_trend": [0.80]  # 60% (Continuous value)
})


In [ ]:
new_data["mood_trend"] = scaler.transform(new_data[["mood_trend"]])

In [ ]:
prediction = model.predict(new_data)

# Convert softmax probabilities to class labels (0, 1, 2, or 3)
predicted_class = np.argmax(prediction)

print(f"Predicted Depression Risk Level: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted Depression Risk Level: 1
